### Assignment 2 Report <br> 
##### Yingnan Zhao id: 260563769

In [1]:
from random import shuffle
import csv
import numpy as np
from sklearn import preprocessing

The function below takes in a file path that stores the data, it will read the data and return a np array contains the data

In [2]:
def CSVfileReader(filePath):
    with open(filePath) as file:
        readCSV = csv.reader(file, delimiter=',')
        data = []
        for row in readCSV:
            inputTemp = []
            for i in range(len(row)):
                if(row[i]!= ''):
                    inputTemp.append(float(row[i]))
            data.append(inputTemp)
    data = np.array(data,dtype='float')
    return data

#### Question 1

The function below will first generate two sets of data according to the provided mean vector and covariance matrix(2000 examples each), for each set of data an extra column will be added at the end, -1 represents class1 and 1 represents class 2. Then it will shuffle the data and pick 30% of each set to form the test set, and 70% of each class to form the training set. They are named DS1_test.csv and DS1_train.csv respectively and stored in the hwk2_datasets_corrected folder.

In [3]:
def generateData1(covPath,meanPath1,meanPath2):
    covData = CSVfileReader(covPath)
    meanData1 = CSVfileReader(meanPath1)
    meanData2 = CSVfileReader(meanPath2)
    class1 = np.random.multivariate_normal(meanData1[0],covData,2000)
    c1 = -1* np.ones((2000,21),dtype='float') #-1 when using u0, c1
    c1[:,:-1] = class1
    class2 = np.random.multivariate_normal(meanData2[0],covData,2000)
    c2 = np.ones((2000,21),dtype='float') #1 when using u1, c2
    c2[:,:-1]= class2
    np.random.shuffle(c1)
    np.random.shuffle(c2)
    testSet = []
    trainSet = []
    for i in range(2000):
        if (i<600):
            testSet.append(c1[i])
            testSet.append(c2[i])
        else:
            trainSet.append(c1[i])
            trainSet.append(c2[i])
    shuffle(testSet)
    shuffle(trainSet)
    with open('hwk2_datasets_corrected\\DS1_test.csv', 'w', newline='') as file:
        csvWriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for j in range(len(testSet)):
            csvWriter.writerow(testSet[j])
    with open('hwk2_datasets_corrected\\DS1_train.csv', 'w', newline='') as file:
        csvWriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for j in range(len(trainSet)):
            csvWriter.writerow(trainSet[j])

#### Question2

The function below will produce W0 and W1 from the training data, first by calculating the possibility and mean for each class and the covariance matrix. Finally we get w0 and w1 fromthe equation below.<br> <img src="LDAEquation.png">

In [4]:
def LDA(trainingPath):
    X = CSVfileReader(trainingPath)
    sum1 = np.zeros(X[0].shape,dtype='float')
    sum2 = np.zeros(X[0].shape,dtype='float')
    N0 = 0
    N1 = 0
    for i in range(len(X)):
        if (X[i][len(X[0])-1]==-1):
            sum1 = np.add(sum1,X[i])
            N0 +=1
        elif (X[i][len(X[0])-1]==1):
            sum2 = np.add(sum2,X[i])
            N1 +=1
    mean1 = np.divide(sum1,N0)
    mean2 = np.divide(sum2,N1)
    mean1 = mean1[:-1].reshape(1,-1)
    mean2 = mean2[:-1].reshape(1,-1)
    P1 = N0/len(X)
    P2 = N1/len(X)
    sum = np.zeros((len(mean1[0]),len(mean1[0])))
    for i in range(len(X)):
        step1 = np.subtract(X[i][:-1],mean1)
        step2 = np.subtract(X[i][:-1],mean2)
        step3 = np.dot(np.transpose(step1),step1)
        step4 = np.dot(np.transpose(step2),step2)
        sum = np.add(np.add(step3,step4),sum)

    cov = np.divide(sum,N0+N1)
    covInverse = np.linalg.inv(cov)
    w1 = np.dot(covInverse,np.transpose(np.subtract(mean1,mean2)))
    term1 = 1/2*np.dot(np.dot(mean1,covInverse),np.transpose(mean1))
    term2 = 1/2*np.dot(np.dot(mean2,covInverse),np.transpose(mean2))
    w0 = np.subtract(term2,term1) + np.log(P1)- np.log(P2)
    w = [w0,w1]
    print('parameter learnt')
    print('w0 = '+str(w0[0][0]))
    print('w1 = '+str(w1))
    return w

The function below will calculate the use the LDA classifier by using w0 and w1 produced by last function and calculate the expected class of each data from a data set and compare with the correct class. The accruacy, precision, recall and F1 measure will be produced.

In [5]:
def testLDA (testFilePath,w):
    X = CSVfileReader(testFilePath)
    result = []
    TPTN = 0.0
    TPFP = 0
    TNFN = 0
    TP =0
    TN =0
    w0 = w[0]
    w1 = w[1]
    for i in range(len(X)):
        ans = w0[0][0] + np.dot(X[i][:-1],w1)
        if (ans>0):
            result.append(-1)
            if (result[i] == X[i][len(X[0]) - 1]):
                TPTN += 1
                TN +=1
            TNFN +=1
        else:
            result.append(1)
            if (result[i] == X[i][len(X[0]) - 1]):
                TPTN += 1
                TP +=1
            TPFP +=1
    accuracy = TPTN/(TPFP+TNFN)
    precision = TP/TPFP
    recall = TP/(TP+TNFN-TN)
    print('treating class 2 as positive')
    print('Accuracy = '+str(accuracy))
    print('Precision = '+str(precision))
    print('Recall = '+str(recall))
    print('F1 Measure = '+ str(2*precision*recall/(precision+recall)))

Run the test on data set one.

In [6]:
w = LDA('hwk2_datasets_corrected\\DS1_train.csv')
testLDA('hwk2_datasets_corrected\\DS1_test.csv',w)

parameter learnt
w0 = 2.01239190359
w1 = [[ 1.05511023]
 [-0.63276528]
 [-0.41290599]
 [-0.22683658]
 [-0.72655371]
 [-0.29721443]
 [ 1.24715225]
 [-1.78586254]
 [-2.14790584]
 [ 0.66975918]
 [-0.95714165]
 [-0.90663457]
 [ 1.15530344]
 [ 0.96745546]
 [-0.42961878]
 [ 0.97122599]
 [ 2.16524577]
 [-0.50713488]
 [-0.04769346]
 [-0.37327559]]
treating class 2 as positive
Accuracy = 0.955
Precision = 0.9595959595959596
Recall = 0.95
F1 Measure = 0.9547738693467336


As shown above, for DS1 the LDA model yields decent result.

#### Question 3

The function will first calculate the distance between the test example and all the samples in the training dataset using the equation below. <br> <img src="KNNdistance.png"> then it will find the k nearest neighbors and assign the input x to the class that is most common in the k neighbors.  

In [7]:
def kNN (trainX,inputX,k):

    distanceSet = []
    for i in range (len(trainX)):
        sum = 0
        for j in range(len(inputX)-1):
            sum += np.square(trainX[i][j]-inputX[j])
        distanceSet.append([np.sqrt(sum),i,trainX[i][len(trainX[0])-1]])
    distanceSet = sorted(distanceSet)
    neighbors = []
    for i in range (k):
        neighbors.append(distanceSet[i][2])
    result = np.sum(neighbors)
    if (result>0):
        return 1
    else:
        return -1

The function below will use the KNN classifier to produce the class for each data sample in the test set and compare the result with the actual class of that sample, it will find the k that yields the best F1 score.Before applying the classifier, the data is normalized by the sklearn preprocessing normalize function. 

In [19]:
def testKNN (trainingSet, testingSet):
    X = CSVfileReader(trainingSet)
    X[:, :-1] = preprocessing.normalize(X[:, :-1], axis=0)
    XTest = CSVfileReader(testingSet)
    XTest[:,:-1] = preprocessing.normalize(XTest[:,:-1],axis=0)
    maxF1 = 0
    maxK = 0
    for j in range (1,21,2):
        TPTN = 0.0
        TPFP = 0
        TNFN = 0
        TP = 0
        TN = 0
        for i in range(len(XTest)):
            ans = kNN(X,XTest[i],j)
            if (ans>0):
                if (ans == X[i][len(X[0]) - 1]):
                    TPTN += 1
                    TN +=1
                TNFN +=1
            else:
                if (ans == X[i][len(X[0]) - 1]):
                    TPTN += 1
                    TP +=1
                TPFP +=1
        precision = TP / TPFP
        recall = TP / (TP + TNFN - TN)
        F1 = 2 * precision * recall / (precision + recall)
        print('k = '+str(j))
        print('F1 Measure = ' + str(F1))
        if(F1 > maxF1):
            maxF1 = F1
            maxK = j
    TPTN = 0.0
    TPFP = 0
    TNFN = 0
    TP = 0
    TN = 0
    #run again the best case
    for i in range(len(XTest)):
        ans = kNN(X, XTest[i], maxK)
        if (ans > 0):
            if (ans == X[i][len(X[0]) - 1]):
                TPTN += 1
                TN += 1
            TNFN += 1
        else:
            if (ans == X[i][len(X[0]) - 1]):
                TPTN += 1
                TP += 1
            TPFP += 1
    accuracy = TPTN/(TPFP+TNFN)
    precision = TP/TPFP
    recall = TP/(TP+TNFN-TN)
    print('treating class 2 as positive')
    print('the Best K is ' + str(maxK))
    print('Accuracy = '+str(accuracy))
    print('Precision = '+str(precision))
    print('Recall = '+str(recall))
    print('F1 Measure = '+ str(2*precision*recall/(precision+recall)))

Run the test to find the best k

In [20]:
testKNN('hwk2_datasets_corrected\\DS1_train.csv','hwk2_datasets_corrected\\DS1_test.csv')

k = 1
F1 Measure = 0.48124428179322964
k = 3
F1 Measure = 0.4918032786885245
k = 5
F1 Measure = 0.4873477038425492
k = 7
F1 Measure = 0.4735376044568246
k = 9
F1 Measure = 0.4626865671641791
k = 11
F1 Measure = 0.46760563380281694
k = 13
F1 Measure = 0.4619718309859155
k = 15
F1 Measure = 0.47457627118644063
k = 17
F1 Measure = 0.4711447492904447
k = 19
F1 Measure = 0.47826086956521735
treating class 2 as positive
the Best K is 3
Accuracy = 0.535
Precision = 0.5113636363636364
Recall = 0.47368421052631576
F1 Measure = 0.4918032786885245


The best number of neighbor is 3, it achieves an accuracy of 53.5% which is slightly better than chance, the classifier is considerably worse than the LDA classifier. And the F1 measure varies slightly when using different k value.

#### Question 4

The function below will first generate the single multivariate Gaussian distributed data for each class according to the input mean and coverance, 6 in total. Then it will randomly pick data from those class according to the probability given (0.1,0.42,0.48) to form the dataset for each of the two class. After that, it will take 30% of the data from each class to form the test set and 70% to form the train set. They are named DS2_test.csv and DS2_train.csv respectively and stored in the hwk2_datasets_corrected folder.

In [11]:
def generateData2(mean_11,mean_12,mean_13,mean_21,mean_22,mean_23,cov_1,cov_2,cov_3):
    mean11 = CSVfileReader(mean_11)
    mean12 = CSVfileReader(mean_12)
    mean13 = CSVfileReader(mean_13)
    mean21 = CSVfileReader(mean_21)
    mean22 = CSVfileReader(mean_22)
    mean23 = CSVfileReader(mean_23)
    cov1 = CSVfileReader(cov_1)
    cov2 = CSVfileReader(cov_2)
    cov3 = CSVfileReader(cov_3)
    class11 = np.random.multivariate_normal(mean11[0], cov1, 2000)
    class12 = np.random.multivariate_normal(mean12[0], cov2, 2000)
    class13 = np.random.multivariate_normal(mean13[0], cov3, 2000)
    class21 = np.random.multivariate_normal(mean21[0], cov1, 2000)
    class22 = np.random.multivariate_normal(mean22[0], cov2, 2000)
    class23 = np.random.multivariate_normal(mean23[0], cov3, 2000)
    np.random.shuffle(class11)
    np.random.shuffle(class12)
    np.random.shuffle(class13)
    np.random.shuffle(class21)
    np.random.shuffle(class22)
    np.random.shuffle(class23)
    c1 = []
    c2 = []
    counter1 =0
    counter2 = 0
    counter3 =0
    for i in range(2000):
        choice = np.random.choice([1,2,3],1,p=[0.1,0.42,0.48])
        if (choice == 1):
            c1.append(class11[i])
            c2.append(class21[i])
            counter1+=1

        elif (choice == 2):
            c1.append(class12[i])
            c2.append(class22[i])
            counter2+=1
        elif (choice == 3):
            c1.append(class13[i])
            c2.append(class23[i])
            counter3+=1
    dataPool1 = -1 * np.ones((2000, 21), dtype='float')  # -1 when using c1
    dataPool2 = np.ones((2000, 21), dtype='float')  # 1 when using c2
    dataPool1[:, :-1] = c1
    dataPool2[:, :-1] = c2
    np.random.shuffle(dataPool1)
    np.random.shuffle(dataPool2)
    testSet = []
    trainSet = []
    for i in range(2000):
        if (i<600):
            testSet.append(dataPool1[i])
            testSet.append(dataPool2[i])
        else:
            trainSet.append(dataPool1[i])
            trainSet.append(dataPool2[i])
    shuffle(testSet)
    shuffle(trainSet)
    with open('hwk2_datasets_corrected\\DS2_test.csv', 'w', newline='') as file:
        csvWriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for j in range(len(testSet)):
            csvWriter.writerow(testSet[j])
    with open('hwk2_datasets_corrected\\DS2_train.csv', 'w', newline='') as file:
        csvWriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for j in range(len(trainSet)):
            csvWriter.writerow(trainSet[j])

#### Question 5

Here we will use the LDA classifier on the dataset 2 produced from question4.

In [13]:
w = LDA('hwk2_datasets_corrected\\DS2_train.csv')
testLDA('hwk2_datasets_corrected\\DS2_test.csv',w)

parameter learnt
w0 = 0.0303414354516
w1 = [[ 0.02985879]
 [-0.02545177]
 [ 0.00645555]
 [ 0.00826951]
 [-0.00987854]
 [ 0.00140088]
 [-0.00376674]
 [-0.02966947]
 [-0.02279957]
 [ 0.01878309]
 [ 0.01552167]
 [-0.0064863 ]
 [-0.04418118]
 [-0.010366  ]
 [ 0.01322158]
 [ 0.00740697]
 [ 0.0111111 ]
 [-0.00113356]
 [ 0.02137036]
 [-0.0070229 ]]
treating class 2 as positive
Accuracy = 0.52
Precision = 0.520066889632107
Recall = 0.5183333333333333
F1 Measure = 0.5191986644407345


As shown above the all of the measures of LDA droped dramatically on dataset 2 which is a mixture of three different Gaussians.

Here we will use the KNN classifier on the dataset 2 produced from question4. And try to find the optimal K.

In [21]:
testKNN('hwk2_datasets_corrected\\DS2_train.csv','hwk2_datasets_corrected\\DS2_test.csv')

k = 1
F1 Measure = 0.5154471544715448
k = 3
F1 Measure = 0.5070892410341952
k = 5
F1 Measure = 0.5063938618925832
k = 7
F1 Measure = 0.5134899912967799
k = 9
F1 Measure = 0.5127304653204565
k = 11
F1 Measure = 0.506896551724138
k = 13
F1 Measure = 0.49826989619377166
k = 15
F1 Measure = 0.5047701647875109
k = 17
F1 Measure = 0.49387040280210165
k = 19
F1 Measure = 0.5
treating class 2 as positive
the Best K is 1
Accuracy = 0.5033333333333333
Precision = 0.5
Recall = 0.5318791946308725
F1 Measure = 0.5154471544715448


As shown above, the optimal k is 1, and it yields similar result as when the KNN classifier being applied to dataset 1.

#### Question 6

There is a huge drop in all measures when applying the LDA classifier to the dataset 2, because dataset 2 is a mixture of 3 gaussian distributions that has different means and different covariance matrix. Thus the dataset violates the key assumptions of LDA classifier, the data is not a gaussian, it is a gaussian mixture, and not all data in each class share the same covariance matrix. However, for the KNN classifier, it does not have any explicit assumptions about the dataset. Thus, the performance measures of the KNN classifier remain relatively stable.